# <center>Predicting and Visualizing the Output Product Molecules of Organic Reactions using Pretrained Transformer Models and RDKit</center>

### Table of contents:

#### 1. [Installing the Packages](#first-bullet)
#### 2. [Data Preprocessing](#second-bullet)
#### 3. [Vocabulary for OpenNMT-py](#third-bullet)
#### 4. [Training the model](#fourth-bullet)
#### 5. [Testing the model](#fifth-bullet)
#### 6. [Performance measures of the model](#sixth-bullet)
#### 7. [Comparison with Previous Work](#seventh-bullete)
#### 8. [Conclusions & Results](#eighth-bullet)

# Installing and Importing the Packages <a class="anchor" id="first-bullet"></a>
We start by installing OpenNMT-py version 2.2.0, which is an open-source (MIT) neural machine translation framework and RDKit, a liabrary to Parse SMILES stings, visualize molecules, etc., followed by pytorch with CUDA to move model tensors.

In [1]:
import rdkit
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
import os
import yaml
import torch #library for deep learning frameworks
from rdkit import RDLogger # optional step to disable warnings of RDkit)
RDLogger.DisableLog('rdApp.*')

# Data Preprocessing <a class="anchor" id="second-bullet"></a>

1. [Loading the Datasets](#firstsub-bullet)
2. [Exploratory Data Analysis](#secondsub-bullet)
3. [Splitting the Dataset](#thirdsub-bullet)
4. [Canonicalizing the Datast](#fourthsub-bullet)
5. [Removing the Atom Maps](#fifthsub-bullet)
6. [Tokenizing the Dataset](#sixthsub-bullet)
7. [Preparing the DataFrames](#seventhsub-bullet)
8. [Shuffling and Saving](#eighthsub-bullet)

### 1. Loading the datasets: Exploratory data analysis <a class="anchor" id="firstsub-bullet"></a>

In [2]:
train = pd.read_csv(r"C:\Users\sayye\OneDrive\Documents\New folder (2)\USPTO 50k dataset\uspto50k_train.csv")
test = pd.read_csv(r"C:\Users\sayye\OneDrive\Documents\New folder (2)\USPTO 50k dataset\uspto50k_test.csv")
val = pd.read_csv(r"C:\Users\sayye\OneDrive\Documents\New folder (2)\USPTO 50k dataset\uspto50k_val.csv")

#a glimpse of the datasets
print(train.head(),'\n\n', val.head(), '\n\n', test.head())

                id  class                      reactants>reagents>production
0       US05849732      6  [NH:1]([CH2:2][CH2:3][CH2:4][CH2:5][C@@H:6]([C...
1  US20120114765A1      2  [C:1](=[O:2])([c:3]1[cH:4][c:5]([N+:6](=[O:7])...
2     US08003648B2      1  [CH3:44][CH2:45][NH:46][CH2:47][CH3:48].[CH:1]...
3     US09045475B2      1  [C:1]([CH2:2][F:3])([CH2:4][F:5])=[O:65].[CH3:...
4     US08188098B2      2  [C:1](=[O:2])([O:3][CH:4]1[CH2:5][CH2:6][CH2:7... 

                 id  class                      reactants>reagents>production
0     US08329716B2      5  [C:1](=[O:2])([C:3]([F:4])([F:5])[F:6])[O:27][...
1       US06051718      5  [CH3:1][C:2]([CH3:3])([CH3:4])[O:5][C:6](=[O:7...
2     US07504410B2      5  [C:1](=[O:2])([C:3]([F:4])([F:5])[F:6])[O:19][...
3       US04960769      5  [CH3:1][C:2]([CH3:3])([CH3:4])[O:5][C:6](=[O:7...
4  US20110092505A1      5  [CH3:1][C:2]([CH3:3])([CH3:4])[O:5][C:6](=[O:7... 

                 id  class                      reactants>reagents>prod

### 2. Exploratory Data Analysis <a class="anchor" id="secondsub-bullet"></a>

In [3]:
print(f'Shape of the train dataset:\t{train.shape}\nShape of the validation dataset:\t{val.shape}\nShape of the test dataset:\t{test.shape}')

Shape of the train dataset:	(40008, 3)
Shape of the validation dataset:	(5001, 3)
Shape of the test dataset:	(5007, 3)


In [4]:
print(f'Info of the train dataset:\t{train.describe()}\nInfo of the validation dataset:\t{val.describe()}\nInfo of the test dataset:\t{test.describe()}')

Info of the train dataset:	              class
count  40008.000000
mean       3.397570
std        2.504438
min        1.000000
25%        1.000000
50%        2.000000
75%        6.000000
max       10.000000
Info of the validation dataset:	             class
count  5001.000000
mean      3.396921
std       2.504121
min       1.000000
25%       1.000000
50%       2.000000
75%       6.000000
max      10.000000
Info of the test dataset:	             class
count  5007.000000
mean      3.399441
std       2.505572
min       1.000000
25%       1.000000
50%       2.000000
75%       6.000000
max      10.000000


In [5]:
print(f'Columns in the train dataset:\t{train.columns}\nColumns in the validation dataset:\t{val.columns}\nColumns in the test dataset:\t{test.columns}')

Columns in the train dataset:	Index(['id', 'class', 'reactants>reagents>production'], dtype='object')
Columns in the validation dataset:	Index(['id', 'class', 'reactants>reagents>production'], dtype='object')
Columns in the test dataset:	Index(['id', 'class', 'reactants>reagents>production'], dtype='object')


In [6]:
train['source'] = 'train'
val['source'] = 'val'
test['source'] = 'test'

df_all = pd.concat([train, val, test], ignore_index=True)
df_all

id  class  \
0           US05849732      6   
1      US20120114765A1      2   
2         US08003648B2      1   
3         US09045475B2      1   
4         US08188098B2      2   
...                ...    ...   
50011  US20050019696A1      2   
50012  US20030139425A1      1   
50013       US05411980      1   
50014       US04426381      6   
50015  US20040067202A1      6   

                           reactants>reagents>production source  
0      [NH:1]([CH2:2][CH2:3][CH2:4][CH2:5][C@@H:6]([C...  train  
1      [C:1](=[O:2])([c:3]1[cH:4][c:5]([N+:6](=[O:7])...  train  
2      [CH3:44][CH2:45][NH:46][CH2:47][CH3:48].[CH:1]...  train  
3      [C:1]([CH2:2][F:3])([CH2:4][F:5])=[O:65].[CH3:...  train  
4      [C:1](=[O:2])([O:3][CH:4]1[CH2:5][CH2:6][CH2:7...  train  
...                                                  ...    ...  
50011  [C:1]([C:2](=[CH2:3])[CH3:4])(=[O:5])[Cl:19].[...   test  
50012  [CH2:1]([c:2]1[cH:3][cH:4][c:5]([F:6])[cH:7][c...   test  
50013  [CH3:7][CH2:8][CH2:9][CH2:10][c:11]1[n:12][nH:...   test  
50014  [O:1]([C:2](=[O:3])[c:4]1[c:5]2[n:6]([c:7]3[cH...   test  
50015  [O:1]([CH2:2][CH2:3][O:4][CH2:5][CH2:6][O:7][C...   test  

[50016 rows x 4 columns]

Since the datsets are open source, they are pretty much clean and don't have null values. We have explored the datasets as per our requirements of the project.

### 3. Splitting the Dataset <a class="anchor" id="thirdsub-bullet"></a>

As shown the dataset has three columns having the id of the reaction, the class, i.e., the type of the reaction, be it elimination, substitution encoded, along with the overall reaction. It's better for the reaction to be split into their components i.e., reactants, reagents and products for further processing.

In [7]:
train[['reactants', 'reagents', 'products']] = train['reactants>reagents>production'].str.split('>', expand = True)
test[['reactants', 'reagents', 'products']] = test['reactants>reagents>production'].str.split('>', expand = True)
val[['reactants', 'reagents', 'products']] = val['reactants>reagents>production'].str.split('>', expand = True)

#checking the columns
print(train.columns, '\n', val.columns, '\n', test.columns)

Index(['id', 'class', 'reactants>reagents>production', 'source', 'reactants',
       'reagents', 'products'],
      dtype='object') 
 Index(['id', 'class', 'reactants>reagents>production', 'source', 'reactants',
       'reagents', 'products'],
      dtype='object') 
 Index(['id', 'class', 'reactants>reagents>production', 'source', 'reactants',
       'reagents', 'products'],
      dtype='object')


### 4. Canonicalizing the Data <a class="anchor" id="fourthsub-bullet"></a>

In [8]:
def canonicalize(smiles): # will raise an Exception if invalid SMILES
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return Chem.MolToSmiles(mol)
    else:
        return ''

train['reactants'] = train['reactants'].apply(canonicalize)
train['reagents'] = train['reagents'].apply(canonicalize)
train['products'] = train['products'].apply(canonicalize)

test['reactants'] = test['reactants'].apply(canonicalize)
test['reagents'] = test['reagents'].apply(canonicalize)
test['products'] = test['products'].apply(canonicalize)

val['reactants'] = val['reactants'].apply(canonicalize)
val['reagents'] = val['reagents'].apply(canonicalize)
val['products'] = val['products'].apply(canonicalize)

### 5. Removing the Atom maps <a class="anchor" id="fifthsub-bullet"></a>

In [ ]:
def remove_atommapping(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        for atom in mol.GetAtoms():
            atom.SetAtomMapNum(0)
        return Chem.MolToSmiles(mol)
    else:
        return ''

train['reactants'] = train['reactants'].apply(remove_atommapping)
train['reagents'] = train['reagents'].apply(remove_atommapping)
train['products'] = train['products'].apply(remove_atommapping)

test['reactants'] = test['reactants'].apply(remove_atommapping)
test['reagents'] = test['reagents'].apply(remove_atommapping)
test['products'] = test['products'].apply(remove_atommapping)

val['reactants'] = val['reactants'].apply(remove_atommapping)
val['reagents'] = val['reagents'].apply(remove_atommapping)
val['products'] = val['products'].apply(remove_atommapping)

### 6. Tokenizing the Dataset <a class="anchor" id="sixthsub-bullet"></a>

To be able to train a language model, we need to split the strings into tokens.

In [ ]:
REGEX_TOKENIZER =  r"(\%\([0-9]{3}\)|\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\||\(|\)|\.|=|#|-|\+|\\|\/|:|~|@|\?|>>?|\*|\$|\%[0-9]{2}|[0-9])"

def tokenize(smiles):
    return ' '.join(smiles)

train['token_reactants'] = train['reactants'].apply(tokenize)
train['token_reagents'] = train['reagents'].apply(tokenize)
train['token_products'] = train['products'].apply(tokenize)

test['token_reactants'] = test['reactants'].apply(tokenize)
test['token_reagents'] = test['reagents'].apply(tokenize)
test['token_products'] = test['products'].apply(tokenize)

val['token_reactants'] = val['reactants'].apply(tokenize)
val['token_reagents'] = val['reagents'].apply(tokenize)
val['token_products'] = val['products'].apply(tokenize)

### 7. Preparing the Data Frames <a class="anchor" id="seventhsub-bullet"></a>

In [ ]:
train_df = pd.DataFrame({'Id': train['id'], 
                         'Class': train['class'], 
                         'Tokenized Reactants': train['token_reactants'],
                         'Tokenized Products': train['token_products'],
                        'Overall Reaction': train['reactants>reagents>production']})
print(f"The training set contains {train_df.shape[0]} reactions.")
train_df.head()

In [ ]:
test_df = pd.DataFrame({'Id': test['id'], 
                        'Class': test['class'],
                        'Tokenized Reactants': test['token_reactants'],
                        'Tokenized Products': test['token_products'],
                       'Overall Reaction': test['reactants>reagents>production']})
print(f"The training set contains {test_df.shape[0]} reactions.")
test_df.head()

In [ ]:
val_df = pd.DataFrame({'Id': val['id'],
                       'Class': val['class'],
                       'Tokenized Reactants': val['token_reactants'],
                       'Tokenized Products': val['token_products'],
                      'Overall Reaction': val['reactants>reagents>production']})
print(f"The training set contains {val_df.shape[0]} reactions.")
val_df.head(7)

### Shuffling and saving the datasets <a class="anchor" id="eighthsub-bullet"></a>

The dataset contains different types of reactions arranged in a ordered manner (as shown the snippet has same type of reaction i.e., 5) hence, without shuffling model might learn patterns that are not generalizable. Shuffling ensures that each training batch has a variety of reaction types, reactants, and complexities. This helps the model learn general rules of reactivity, and avoid overfitting. After shuffling it can be seen the dataset is random from the classof the reaction.

In [ ]:
train_rn = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_rn = test_df.sample(frac=1, random_state=42).reset_index(drop=True)
val_rn = val_df.sample(frac=1, random_state=42).reset_index(drop=True)
train_rn.head(20)

In [ ]:
train_rn['Tokenized Reactants'].to_csv("uspto_50k_train_reactants.txt", index=False, header=False)
train_rn['Tokenized Products'].to_csv("uspto_50k_train_products.txt", index=False, header=False)

test_rn['Tokenized Reactants'].to_csv("uspto_50k_test_reactants.txt", index=False, header=False)
test_rn['Tokenized Products'].to_csv("uspto_50k_test_products.txt", index=False, header=False)

val_rn['Tokenized Reactants'].to_csv("uspto_50k_val_reactants.txt", index=False, header=False)
val_rn['Tokenized Products'].to_csv("uspto_50k_val_products.txt", index=False, header=False)

# Vocabulary for OpenNMT-py <a class="anchor" id="third-bullet"></a>

Working with Open-NMT-py requires to know, what tokens exists (here its sequence of atoms and bonds), how long can the input/out sequence be, where the training, validation data is, and how to handle tokenization to adapt to any language translation.

Source: https://opennmt.net/OpenNMT-py/options/build_vocab.html

In [ ]:
config = {
    'save_data': 'uspto_50k_run',
    'src_vocab': 'uspto_50k_run/vocab.src',
    'tgt_vocab': 'uspto_50k_run/vocab.tgt',
    'overwrite': True,
    'share_vocab': True,
    'data': {
        'corpus-1': {
            'path_src': 'uspto_50k_train_reactants.txt',
            'path_tgt': 'uspto_50k_train_products.txt',
        },
        'valid': {
            'path_src': 'uspto_50k_val_reactants.txt',
            'path_tgt': 'uspto_50k_val_products.txt',
        }
    }
}

# Create folder and save YAML file
folder = 'example_run'
os.makedirs(folder, exist_ok=True)
with open(os.path.join(folder, 'run_config.yaml'), 'w') as f:
    yaml.dump(config, f)

print("run_config.yaml created successfully!")

In [ ]:
os.path.exists("example_run/run_config.yaml") #checking if the config file has been created or not

In [ ]:
!python -m onmt.bin.build_vocab -config example_run/run_config.yaml -n_sample -1

In [ ]:
print(os.path.exists("uspto_50k_run/vocab.src"))  # Should be True
print(os.path.exists("uspto_50k_run/vocab.tgt"))

# Training the model <a class="anchor" id="fourth-bullet"></a>

Building a Transformer model from scratch on CPU can take hours. Considering this project an experimentation, we will create a small transformer model, with hit and trial method to choose the right train_steps and choose the right one. 

```yaml
world_size: 1
gpu_ranks: [0]  # Use CPU by removing or modifying this
train_steps: 20000
save_model: example_run/model
save_checkpoint_steps: 1000
batch_size: 64
valid_steps: 10000
report_every: 100

In [ ]:
!onmt_train -config example_run/run_config.yaml \
-seed 42 \
-batch_type tokens -batch_size 64 \
-accum_count 2 \
-optim adam -adam_beta2 0.998 \
-decay_method noam -warmup_steps 4000 \
-learning_rate 1 \
-label_smoothing 0.0 \
-layers 2 -rnn_size 256 -word_vec_size 256 \
-encoder_type transformer -decoder_type transformer \
-dropout 0.1 -position_encoding \
-share_embeddings \
-transformer_ff 1024 \
-tensorboard True -tensorboard_log_dir log_dir

[INFO] Step 40000/40000; acc: 48.1; ppl: 32.5; xent: 44.56; lr: 0.00038; ...e.

[INFO] Saving checkpoint to example_run/model_step_40000.pt

[INFO] Training complete.

In [ ]:
!tensorboard --logdir log_dir

# Performance measures of the model <a class="anchor" id="fifth-bullet"></a>

In [ ]:
!onmt_translate \
  -model model_step_20000.pt \
  -src path/to/val_src.txt \
  -tgt path/to/val_tgt.txt \
  -output example_run/predictions_val.txt \
  -gpu 0 \
  -beam_size 10 \
  -n_best 5 \
  -max_length 380 \
  -batch_size 64

In [ ]:
import pandas as pd
from rdkit import Chem
from tqdm import tqdm

# Load ground truth and predictions
with open("path/to/val_tgt.txt") as f:
    targets = [line.strip() for line in f]

with open("path/to/val_src.txt") as f:
    precursors = [line.strip() for line in f]

n_best = 5
predictions = [[] for _ in range(n_best)]

with open("example_run/predictions_val.txt") as f:
    for i, line in enumerate(f):
        predictions[i % n_best].append(line.strip())

# Create evaluation DataFrame
df = pd.DataFrame({
    "target": targets,
    "precursors": precursors
})

for i in range(n_best):
    df[f"prediction_{i+1}"] = predictions[i]

# Canonicalize SMILES
def canonicalize(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return Chem.MolToSmiles(mol)
    except:
        return None

for i in range(n_best):
    df[f"canonical_prediction_{i+1}"] = df[f"prediction_{i+1}"].apply(canonicalize)

# Calculate top-n accuracy
def get_rank(row):
    for i in range(n_best):
        if row["target"] == row[f"canonical_prediction_{i+1}"]:
            return i + 1
    return None

tqdm.pandas()
df["rank"] = df.progress_apply(get_rank, axis=1)

# Report top-n accuracy
for i in range(1, n_best + 1):
    correct = df["rank"].apply(lambda x: x is not None and x <= i).sum()
    print(f"Top-{i} Accuracy: {correct / len(df) * 100:.2f}%")

# Report invalid SMILES
for i in range(n_best):
    invalid = df[f"canonical_prediction_{i+1}"].isna().sum()
    print(f"Invalid SMILES in Top-{i+1}: {invalid}")